In [1]:
import pickle
import anndata2ri
import scanpy
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

#This activates the object conversion.
anndata2ri.activate()
%load_ext rpy2.ipython

/tmp/ipykernel_725555/19227154.py:9: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [2]:
#Import needed R libraries.
seurat = importr('Seurat')
base = importr('base')


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

In [3]:
#Read Seurat object into R.
robjects.r('seurat_object = readRDS("/scratch/aoill/projects/heart_transplant/new/merged_spatial_filtered_macro_pass3.rds")')

# Other temp objects that went through this pipeline for annotation
#robjects.r('seurat_object = readRDS("/scratch/aoill/projects/heart_transplant/new/merged_spatial_filtered_macro_c5_pass3.rds")')
#robjects.r('seurat_object = readRDS("/scratch/aoill/projects/heart_transplant/new/merged_spatial_filtered_macro_pass3.rds")')
#robjects.r('seurat_object = readRDS("/scratch/aoill/projects/heart_transplant/new/merged_spatial_filtered_endo_pass2.rds")')
#robjects.r('seurat_object = readRDS("/scratch/aoill/projects/heart_transplant/new/merged_spatial_filtered_mes_pass2.rds")')
#robjects.r('seurat_object = readRDS("/scratch/aoill/projects/heart_transplant/new/merged_spatial_filtered_T.rds")')
#robjects.r('seurat_object = readRDS("/scratch/aoill/projects/heart_transplant/new/merged_spatial_filtered_lymphoid.rds")')
#robjects.r('seurat_object = readRDS("/scratch/aoill/projects/heart_transplant/new/merged_spatial_filtered_meyloid.rds")')
#robjects.r('seurat_object = readRDS("/scratch/aoill/projects/heart_transplant/new/merged_spatial_filtered_imm_pass2.rds")')
#robjects.r('seurat_object = readRDS("/scratch/aoill/projects/heart_transplant/new/merged_spatial_filtered_card_pass1.rds")')
#robjects.r('seurat_object = readRDS("/scratch/aoill/projects/heart_transplant/new/merged_spatial_filtered_endo_pass1.rds")')
#robjects.r('seurat_object = readRDS("/scratch/aoill/projects/heart_transplant/new/merged_spatial_filtered_mes_pass1.rds")')
#robjects.r('seurat_object = readRDS("/scratch/aoill/projects/heart_transplant/new/merged_spatial_filtered_imm_pass1.rds")')
#robjects.r('seurat_object = readRDS("/scratch/aoill/projects/heart_transplant/xenium_spatial_cells_only_filtered.rds")')

<rpy2.robjects.methods.RS4 object at 0x15549c30acc0> [RTYPES.S4SXP]
R classes: ('Seurat',)

In [4]:
#Merge layers because Seurat object versioning is wild. Convert Seurat object to Anndata.
robjects.r('seurat_object = JoinLayers(seurat_object)')
adata = robjects.r('as.SingleCellExperiment(seurat_object)')

/opt/conda/envs/rpy2/lib/python3.10/site-packages/anndata2ri/_r2py.py:132: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  return AnnData(exprs, obs, var, uns, obsm, layers=layers)


In [5]:
#Check to make sure conversion worked and possibly cast counts to array if necessary.
# Might not need to run, if the data was already in array format
adata.X = adata.X.toarray()

In [6]:
adata.X

array([[0., 0., 0., ..., 0., 5., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 2., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 2., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 2., 0., 0.]], dtype=float32)

In [7]:
del adata.layers

In [8]:
#Write anndata to pickle file. Next move to the clustering script.
with open("merged_spatial_filtered", "wb") as adata_object:
    pickle.dump(adata, adata_object)

In [9]:
adata

AnnData object with n_obs × n_vars = 5849 × 350
    obs: 'orig.ident', 'nCount_Xenium', 'nFeature_Xenium', 'nCount_BlankCodeword', 'nFeature_BlankCodeword', 'nCount_ControlCodeword', 'nFeature_ControlCodeword', 'nCount_ControlProbe', 'nFeature_ControlProbe', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'genomic_control_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count', 'segmentation_method', 'Sample', 'slide', 'adj_x_centroid', 'adj_y_centroid', 'percent_BlankCodeword', 'percent_ControlCodeword', 'percent_ControlProbe', 'ident'
    obsm: 'SP'

In [ ]:
# Immediately go to clustering script